In [9]:
from officelib.xllib import *
from officelib.const import xlDown, xlToRight, xlA1
from hello.pid.ag_pid.agpid import *

In [10]:
import subprocess
subprocess.call("tskill.exe excel")

0

In [11]:
xl = Excel()
for rf in xl.RecentFiles:
    if rf.Name.endswith("80L Test Compiled Data.xlsx"):
        wb = rf.Open()
        break
else:
    raise ValueError("Not Found")

xl.Selection.Value

ws = wb.Worksheets("Sheet1")

cells = ws.Cells


In [12]:
def make_summary(p, i, d, start_mode, st, sp, vol, new_name):
    paste_cell.Offset(1, 1).Value = n
    paste_cell.Offset(1, 2).Value = p
    paste_cell.Offset(1, 3).Value = i
    paste_cell.Offset(1, 4).Value = d
    paste_cell.Offset(1, 5).Value = start_mode
    paste_cell.Offset(1, 6).Value = st
    paste_cell.Offset(1, 7).Value = sp
    paste_cell.Offset(1, 8).Value = vol
    paste_cell.Offset(1, 9).Value = new_name

    
def get_value(r, c, f="var"):
    v = cell.Offset(r, c).Value
    if v is None:
        raise ValueError("No value for %s" % f)
    return v

first = cells.Range("A1")
cell = first
ws2 = wb.Worksheets("Sheet2")
ws2.UsedRange.Clear()
paste_cell = ws2.Cells.Range("B3")
header = ws2.Cells.Range(paste_cell.Offset(0, 1), paste_cell.Offset(0, 9))
header.Value = [("Test #", "PGain", "Itime", "DTime", "Start Mode", "StartPV", "SP", "Volume", "Name")]
n = 1
runner = PIDRunner((), "compiled_data.xlsx")
while True:
    v = cell.Value
    if not v:
        break
    name, start, setpoint = v.split()
    _, st = start.split(":")
    _, sp = setpoint.split(":")
    print("Exporting data for", name, start, setpoint)
    p = get_value(2, 2, "pgain")
    i = get_value(3, 2, "itime")
    d = get_value(4, 2, "dtime")
    vol = get_value(4, 4, "volume")
    new_name = v + " Vol:%d" % vol
    start_mode, end_mode = name.split("To")
    make_summary(p, i, d, start_mode, st, sp, vol, new_name)
    paste_cell = paste_cell.Offset(2, 1)
    
    if name == "AutoToAuto":
        klass = TempAutoToAutoTest
    elif name == "OffToAuto":
        klass = TempOffToAutoTest
    elif name == "ManToAuto":
        klass = TempManToAutoTest
    elif name == "ManToMan":
        klass = TempManToManTest
    else:
        raise ValueError(name)
    t = klass(p, i, d, st, sp, "", None, new_name)
    
    tl_data = cell.Offset(7, 1)
    br_data = tl_data.End(xlDown).End(xlToRight)
    t.data = cells.Range(tl_data, br_data).Value
    t.metadata = {'Volume': vol, "Beta": 1}
    
    runner.add_test(t)
    n += 1
    cell = cell.Offset(1, 6)
    
runner._init_xl()
with screen_lock(runner._xl):
    runner.plotall()
    runner.chartbypid()
runner._wb.Save()
print("Done")

Exporting data for AutoToAuto Start:37.0 SP:35.8
Exporting data for AutoToAuto Start:35.8 SP:37.0
Exporting data for AutoToAuto Start:37.0 SP:35.8
Exporting data for AutoToAuto Start:35.8 SP:37.0
Exporting data for OffToAuto Start:26.0 SP:37.0
Exporting data for AutoToAuto Start:37.0 SP:35.0
Exporting data for AutoToAuto Start:35.0 SP:37.0
Exporting data for OffToAuto Start:23 SP:37.0
Exporting data for AutoToAuto Start:37.0 SP:39.0
Exporting data for AutoToAuto Start:39.0 SP:37.0
Exporting data for OffToAuto Start:26 SP:37
Exporting data for AutoToAuto Start:37.0 SP:39.0
Exporting data for AutoToAuto Start:39.0 SP:37.0
Exporting data for AutoToAuto Start:37.0 SP:35.8
Exporting data for AutoToAuto Start:35.8 SP:37.0
Exporting data for AutoToAuto Start:37.0 SP:35.0
Exporting data for AutoToAuto Start:35.0 SP:37.0
Exporting data for ManToAuto Start:37.0 SP:35.8
Exporting data for ManToAuto Start:35.8 SP:37.0
Exporting data for ManToAuto Start:37.0 SP:35.0
Exporting data for ManToAuto Sta